In [46]:
import os
import shutil
from multiprocessing import Pool

import librosa
import pandas as pd
import soundfile as sf

from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm

tqdm.pandas()

lang = 'ta'

In [3]:
data_dir = f"/home/gokulkarthikk/datasets/googletts/{lang}/raw" # update the path
data_dir_new = f"/home/gokulkarthikk/datasets/googletts/{lang}/processed" # update the path

# Process to LJSpeech Format

In [4]:
if not os.path.exists(data_dir_new):
    os.makedirs(data_dir_new)

In [16]:
shutil.copytree(f'{data_dir}/male', f'{data_dir_new}/wavs')

'/home/gokulkarthikk/datasets/googletts/ta/processed/wavs'

In [18]:
shutil.copytree(f'{data_dir}/female', f'{data_dir_new}/wavs', dirs_exist_ok=True)

'/home/gokulkarthikk/datasets/googletts/ta/processed/wavs'

In [20]:
os.remove(f'{data_dir_new}/wavs/LICENSE')
os.remove(f'{data_dir_new}/wavs/line_index.tsv')

In [10]:
metadata_male_fp = f"{data_dir}/male/line_index.tsv"
metadata_male = pd.read_csv(metadata_male_fp, sep='\t', header=None, names=['id', 'text'])
metadata_male['speaker'] = metadata_male['id'].str.rsplit('_', 1).str[0]
metadata_male['gender'] = 'male'
print(metadata_male.shape)
metadata_male.head()

(1956, 4)


,id,text,speaker,gender
0,tag_09162_01279273055,அவர்களின் படங்களின் டீஸருக்கு கிடைக்கும் வரவேற...,tag_09162,male
1,tag_01515_01483902699,பாம்பன் ரயில் பாலத்துக்கு வெடிகுண்டு மிரட்டல்,tag_01515,male
2,tag_03777_01709938953,ஜமேக்கா வன்முறைகளில் முப்பது பேர் உயிரிழப்பு,tag_03777,male
3,tag_01181_00836855113,கிழக்குப் பல்கலைக்கழத்தில் போருக்குப் பின் அறி...,tag_01181,male
4,tag_03191_01070187495,நான் நடிகை என்பதற்காக இருவத்தி நான்கு மணி நேரம...,tag_03191,male


In [11]:
metadata_female_fp = f"{data_dir}/female/line_index.tsv"
metadata_female = pd.read_csv(metadata_female_fp, sep='\t', header=None, names=['id', 'text'])
metadata_female['speaker'] = metadata_female['id'].str.rsplit('_', 1).str[0]
metadata_female['gender'] = 'female'
print(metadata_female.shape)
metadata_female.head()

(2335, 4)


,id,text,speaker,gender
0,taf_02345_00348037167,ஆஸ்த்ரேலியப் பெண்ணுக்கு முப்பத்தி மூன்று ஆண்டு...,taf_02345,female
1,taf_07049_00155837462,ஸ்ரீரங்கம் கோவிலில் வெடிகுண்டு மிரட்டல்,taf_07049,female
2,taf_09705_01218130267,உங்களுடைய உணவுக் கட்டுப்பாட்டைச் சொன்னால் மற்ற...,taf_09705,female
3,taf_03219_00712757493,ரஹானே மட்டும் ஆறுதலாக முப்பத்தி ஆறு ரன்கள் குவ...,taf_03219,female
4,taf_00008_01305524612,மனோரமாவிற்கு பூபதி என்ற மகன் உள்ளார்,taf_00008,female


In [12]:
metadata = pd.concat([metadata_male, metadata_female]).reset_index(drop=True)
metadata

,id,text,speaker,gender
0,tag_09162_01279273055,அவர்களின் படங்களின் டீஸருக்கு கிடைக்கும் வரவேற...,tag_09162,male
1,tag_01515_01483902699,பாம்பன் ரயில் பாலத்துக்கு வெடிகுண்டு மிரட்டல்,tag_01515,male
2,tag_03777_01709938953,ஜமேக்கா வன்முறைகளில் முப்பது பேர் உயிரிழப்பு,tag_03777,male
3,tag_01181_00836855113,கிழக்குப் பல்கலைக்கழத்தில் போருக்குப் பின் அறி...,tag_01181,male
4,tag_03191_01070187495,நான் நடிகை என்பதற்காக இருவத்தி நான்கு மணி நேரம...,tag_03191,male
...,...,...,...,...
4286,taf_07910_01765928277,அட்லாண்டிஸ் விண்ணோடம் தனது கடைசிப் பயணத்தை ஆரம...,taf_07910,female
4287,taf_07352_02032741859,ஜூன் ஐந்து சுகாதார தினம்,taf_07352,female
4288,taf_08213_00525564801,நாடாளுமன்றக் கூட்டுக்குழு அமைக்கப்படும்,taf_08213,female
4289,taf_05572_02054420639,இரண்டாயிரத்து இரண்டு வன்முறை தொடர்பான ஆவணங்களை...,taf_05572,female


In [13]:
metadata.to_csv(f'{data_dir_new}/metadata.csv', sep='|', index=False, header=False)

# Resampling

In [15]:
if not os.path.exists(f'{data_dir_new}/wavs-20k'):
    os.makedirs(f'{data_dir_new}/wavs-20k')

In [23]:
def resample_file(func_args):
    fp_src, fp_dst, output_sr = func_args
    y, sr = librosa.load(fp_src, sr=output_sr)
    sf.write(fp_dst, y, sr)

In [21]:
fps_src = [f'{data_dir_new}/wavs/{fn}' for fn in tqdm(os.listdir(f'{data_dir_new}/wavs'))]
fps_dst = [f'{data_dir_new}/wavs-20k/{fn}' for fn in tqdm(os.listdir(f'{data_dir_new}/wavs'))]
srs = [22050] * len(fps_src)

  0%|          | 0/4291 [00:00<?, ?it/s]

  0%|          | 0/4291 [00:00<?, ?it/s]

In [24]:
audio_files = list(zip(fps_src, fps_dst, srs))
with Pool(processes=16) as p:
    with tqdm(total=len(fps_src)) as pbar:
        for i, _ in enumerate(p.imap_unordered(resample_file, audio_files)):
            pbar.update()

  0%|          | 0/4291 [00:00<?, ?it/s]

In [ ]:
# audio_files = list(zip(fps_src, fps_dst, srs))
# for audio_file in tqdm(audio_files):
#     resample_file(audio_file)

# Filter data

In [55]:
metadata = pd.read_csv(f'{data_dir_new}/metadata.csv', sep='|', names=['id', 'text', 'speaker', 'gender'])
metadata.head()

,id,text,speaker,gender
0,tag_09162_01279273055,அவர்களின் படங்களின் டீஸருக்கு கிடைக்கும் வரவேற...,tag_09162,male
1,tag_01515_01483902699,பாம்பன் ரயில் பாலத்துக்கு வெடிகுண்டு மிரட்டல்,tag_01515,male
2,tag_03777_01709938953,ஜமேக்கா வன்முறைகளில் முப்பது பேர் உயிரிழப்பு,tag_03777,male
3,tag_01181_00836855113,கிழக்குப் பல்கலைக்கழத்தில் போருக்குப் பின் அறி...,tag_01181,male
4,tag_03191_01070187495,நான் நடிகை என்பதற்காக இருவத்தி நான்கு மணி நேரம...,tag_03191,male


In [56]:
metadata['text_length'] = metadata['text'].str.len()
print(len(metadata))
metadata = metadata[~metadata['text_length'].isna()]
metadata['text_length'] = metadata['text_length'].astype(int)
print(len(metadata))
metadata.head()

4291
4284


,id,text,speaker,gender,text_length
0,tag_09162_01279273055,அவர்களின் படங்களின் டீஸருக்கு கிடைக்கும் வரவேற...,tag_09162,male,66
1,tag_01515_01483902699,பாம்பன் ரயில் பாலத்துக்கு வெடிகுண்டு மிரட்டல்,tag_01515,male,45
2,tag_03777_01709938953,ஜமேக்கா வன்முறைகளில் முப்பது பேர் உயிரிழப்பு,tag_03777,male,44
3,tag_01181_00836855113,கிழக்குப் பல்கலைக்கழத்தில் போருக்குப் பின் அறி...,tag_01181,male,86
4,tag_03191_01070187495,நான் நடிகை என்பதற்காக இருவத்தி நான்கு மணி நேரம...,tag_03191,male,80


In [57]:
def get_duration(id_):
    fp = f'{data_dir_new}/wavs-20k/{id_}.wav'
    y, sr = librosa.load(fp)
    duration = librosa.get_duration(y=y, sr=sr)
    return duration

In [58]:
metadata['audio_duration'] = metadata['id'].progress_apply(get_duration)
print(len(metadata))
metadata = metadata[metadata['audio_duration']>0]
print(len(metadata))
metadata.head()

  0%|          | 0/4284 [00:00<?, ?it/s]

4284
4284


,id,text,speaker,gender,text_length,audio_duration
0,tag_09162_01279273055,அவர்களின் படங்களின் டீஸருக்கு கிடைக்கும் வரவேற...,tag_09162,male,66,5.120000
1,tag_01515_01483902699,பாம்பன் ரயில் பாலத்துக்கு வெடிகுண்டு மிரட்டல்,tag_01515,male,45,4.096009
2,tag_03777_01709938953,ஜமேக்கா வன்முறைகளில் முப்பது பேர் உயிரிழப்பு,tag_03777,male,44,4.693333
3,tag_01181_00836855113,கிழக்குப் பல்கலைக்கழத்தில் போருக்குப் பின் அறி...,tag_01181,male,86,8.960000
4,tag_03191_01070187495,நான் நடிகை என்பதற்காக இருவத்தி நான்கு மணி நேரம...,tag_03191,male,80,5.973333


In [62]:
metadata['audio_duration'].sum()/(60*60)

7.069106160241875

In [59]:
metadata.describe()

,text_length,audio_duration
count,4284.000000,4284.000000
mean,60.683707,5.940425
std,23.679106,1.886870
min,7.000000,1.706667
25%,44.000000,4.608027
50%,59.000000,5.717370
75%,76.000000,7.082676
max,160.000000,17.152018


In [63]:
metadata.to_csv(f'{data_dir_new}/metadata.csv', sep='|', index=False, header=False)

# Make train-test splits

In [64]:
df = pd.read_csv(f'{data_dir_new}/metadata.csv', sep='|', names=['id', 'text', 'speaker', 'gender', 'text_length', 'audio_duration'])
print(df.shape)
df.head()

(4284, 6)


,id,text,speaker,gender,text_length,audio_duration
0,tag_09162_01279273055,அவர்களின் படங்களின் டீஸருக்கு கிடைக்கும் வரவேற...,tag_09162,male,66,5.120000
1,tag_01515_01483902699,பாம்பன் ரயில் பாலத்துக்கு வெடிகுண்டு மிரட்டல்,tag_01515,male,45,4.096009
2,tag_03777_01709938953,ஜமேக்கா வன்முறைகளில் முப்பது பேர் உயிரிழப்பு,tag_03777,male,44,4.693333
3,tag_01181_00836855113,கிழக்குப் பல்கலைக்கழத்தில் போருக்குப் பின் அறி...,tag_01181,male,86,8.960000
4,tag_03191_01070187495,நான் நடிகை என்பதற்காக இருவத்தி நான்கு மணி நேரம...,tag_03191,male,80,5.973333


In [65]:
df_train, df_test = train_test_split(df, test_size=100, stratify=df['speaker'], random_state=0)
print(len(df_train), len(df_test))

4184 100


In [66]:
df_train.to_csv(f'{data_dir_new}/metadata_train.csv', sep='|', index=False, header=False)
df_test.to_csv(f'{data_dir_new}/metadata_test.csv', sep='|', index=False, header=False)